In [1]:
import os
import pandas as pd
import numpy as np

In [11]:
import sys

sys.path.append(os.path.dirname(os.getcwd()))

from utils.stock_assessments import *
from utils.species_landings import *
from utils.stock_landings import *
from utils.stock_weights import *

In [4]:
clean_data_dir = os.path.join(os.path.dirname(os.getcwd()), "output", "clean_data")
agg_tables_dir = os.path.join(
    os.path.dirname(os.getcwd()), "output", "aggregate_tables"
)

In [61]:
stock_landings = pd.read_excel(os.path.join(clean_data_dir, "stock_landings.xlsx"))

In [63]:
weights = pd.read_excel(os.path.join(clean_data_dir, "stock_weights.xlsx"))

In [64]:
weights[weights["ASFIS Scientific Name"] == "Halichoerus grypus"]

,Area,ASFIS Scientific Name,Location,Weight 1,Weight 2,Location_new,Normalized Weight
42,21,Halichoerus grypus,Atlantic,NaN,1.0,Atlantic,1.0


In [54]:
check = weights[
    (weights["ASFIS Scientific Name"] == "Parapenaeus longirostris")
    & (weights["Area"] == 34)
]

check_group = check.groupby(["Area", "ASFIS Scientific Name"])

In [65]:
species_landings = pd.read_excel(os.path.join(clean_data_dir, "species_landings.xlsx"))

In [67]:
species_landings[species_landings[2021] == 0]

,Area,ASFIS Scientific Name,Location,1950,1951,1952,1953,1954,1955,1956,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,21,Amblyraja radiata,Gulf of Maine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
23,21,Cucumaria frondosa,Areas B and C in the Gaspé Peninsula,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
24,21,Cucumaria frondosa,"Maritimes Region, and SWNB Sea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
25,21,Cucumaria frondosa,NAFO Subdivision 3Ps,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
42,21,Halichoerus grypus,Atlantic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,Deep Sea,Beryx decadactylus,Corner Rise seamounts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
2465,Deep Sea,Pentaceros wheeleri,NPFC: N Pacific,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0
2467,Deep Sea,Scomber australasicus,NPFC: N Pacific,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3324.0,12656.0,2471.0,8889.00,22244.00,0.00,0.00,0.0,0.00,0.0
2569,Sharks,Isurus oxyrinchus,Pacific North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,0.0,0.0,0.00,1.00,0.00,0.00,0.0,0.00,0.0


In [49]:
def compute_weights(group):
    # See if weight 2 should be used
    if all(
        group["Weight 1"].apply(lambda x: isinstance(x, str) or pd.isna(x) or x == 0)
    ):
        if all(group["Weight 2"].isna()) or all(group["Weight 2"] == 0):
            return pd.Series(1 / len(group), index=group.index)
        else:
            group["Weight 2"] = group["Weight 2"].fillna(1)
            return group["Weight 2"] / group["Weight 2"].sum()

    # Set base value of Weight 1 to 0.001
    # All stocks with species landings should get non-zero catch
    zero_mask = group["Weight 1"] == 0
    group.loc[zero_mask, "Weight 1"] = 1e-3

    # Get all rows with valid entries for Weight 1
    val = group[
        group["Weight 1"].apply(
            lambda x: isinstance(x, (int, float)) and not pd.isna(x)
        )
    ]

    # Rows with no assigned secondary weight get lowest value of 1
    group["Weight 2"] = group["Weight 2"].fillna(1)

    for idx, row in group.iterrows():
        if isinstance(row["Weight 1"], str) or pd.isna(row["Weight 1"]):
            # If priority weight is missing
            if sum(val["Weight 2"] == row["Weight 2"]) > 0:
                # Check if there is a stock which has priority weight with equal secondary weight
                # Use the mean priority weight from all stocks with equal secondary weight
                group.loc[idx, "weight"] = val[val["Weight 2"] == row["Weight 2"]][
                    "Weight 1"
                ].mean()
            else:
                # Find the row(s) which secondary weight closest to this row's secondary weight
                w = min(
                    val["Weight 2"].unique(), key=lambda x: abs(x - row["Weight 2"])
                )
                group.loc[idx, "weight"] = val[val["Weight 2"] == w]["Weight 1"].mean()
        else:
            group.loc[idx, "weight"] = row["Weight 1"]
            print(idx, group.loc[idx, "weight"])

    return group["weight"] / group["weight"].sum()

In [69]:
input_dir = os.path.join(os.path.dirname(os.getcwd()), "input")

In [70]:
# Retrieve fishstat and ASFIS data for NEI landings
fishstat = pd.read_csv(os.path.join(input_dir, "global_capture_production.csv"))
mappings = get_asfis_mappings(input_dir, "ASFIS_sp_2024.csv")
asfis = mappings["ASFIS"]
code_to_scientific = dict(zip(asfis["Alpha3_Code"], asfis["Scientific_Name"]))
scientific_to_name = mappings["ASFIS Scientific Name to ASFIS Name"]

fishstat = format_fishstat(fishstat, code_to_scientific)
fishstat["ASFIS Name"] = fishstat["ASFIS Scientific Name"].map(scientific_to_name)

In [84]:
def compute_weighted_percentages(
    stock_landings,
    fishstat=None,
    key="Area",
    tuna_location_to_area={},
    year=2021,
    landings_key="Stock Landings 2021",
):
    data = stock_landings.copy()

    if tuna_location_to_area and key == "Area":
        # Add the Tuna back into the areas from which they came
        # One tuna assessment corresponds to multiple
        # assessments added back into the area with the same status
        # as original assessment and landing specific to that area
        sn = "ASFIS Scientific Name"
        tuna_in_areas = pd.DataFrame()
        for idx, row in data[data["Area"] == "Tuna"].iterrows():
            areas = tuna_location_to_area[row["Location"]]

            for area in areas:
                tuna_capture = fishstat[
                    (fishstat["Area"] == area) & (fishstat[sn] == row[sn])
                ][year].sum()
                if tuna_capture > 0:
                    tuna_in_area = pd.DataFrame(
                        {
                            "Area": area,
                            "ASFIS Scientific Name": row[sn],
                            "Status": row["Status"],
                            landings_key: tuna_capture,
                        },
                        index=[len(tuna_in_areas)],
                    )
                    tuna_in_areas = pd.concat([tuna_in_areas, tuna_in_area])

        # Add the area specific tuna rows, and remove the Tuna category
        data = data[~(data["Area"] == "Tuna")]
        data = pd.concat([data, tuna_in_areas]).reset_index(drop=True)

    # Group by key and Status to aggregate data
    group = data.groupby([key, "Status"])[landings_key].sum().unstack(fill_value=0)

    # Add a "Global" aggregation row
    global_totals = group.sum(axis=0)
    global_totals.name = "Global"
    group = pd.concat([group, global_totals.to_frame().T])

    # Calculate total landings per group
    total_landings = group.sum(axis=1).to_frame(name="Total Landings (Mt)")

    # Ensure required columns exist before computations
    for col in ["M", "U", "O"]:
        if col not in group.columns:
            group[col] = 0  # Add missing columns to avoid KeyErrors

    # Compute total sustainable and unsustainable landings
    total_landings["Sustainable (Mt)"] = (group["M"] + group["U"]) / 1e6
    total_landings["Unsustainable (Mt)"] = group["O"] / 1e6
    total_landings["MSF (Mt)"] = group["M"] / 1e6
    total_landings["U (Mt)"] = group["U"] / 1e6
    total_landings["O (Mt)"] = group["O"] / 1e6

    # Ensure no division by zero
    wp = group.div(group.sum(axis=1).replace(0, 1), axis=0) * 100

    # Compute weighted percentages
    wp["Sustainable (%)"] = wp["M"] + wp["U"]
    wp["Unsustainable (%)"] = wp["O"]
    wp.rename(columns={"U": "U (%)", "M": "MSF (%)", "O": "O (%)"}, inplace=True)

    if key == "Area" and "48,58,88" not in total_landings.index:
        new_row = pd.DataFrame(
            {
                "Total Landings (Mt)": 0,
                "Sustainable (Mt)": 0,
                "Unsustainable (Mt)": 0,
                "MSF (Mt)": 0,
                "U (Mt)": 0,
                "O (Mt)": 0,
            },
            index=["48,58,88"],
        )
        total_landings = pd.concat([total_landings, new_row])

    # Organize and rename columns
    total_landings = total_landings[
        ["U (Mt)", "MSF (Mt)", "O (Mt)", "Sustainable (Mt)", "Unsustainable (Mt)"]
    ]
    wp = wp[["U (%)", "MSF (%)", "O (%)", "Sustainable (%)", "Unsustainable (%)"]]

    # Combine totals and percentages
    result = pd.concat(
        [total_landings, wp],
        axis=1,
        keys=["Total Landings", "Weighted % by Landings"],
    )

    result.index.name = key

    return result

In [73]:
import json

In [74]:
with open(os.path.join(input_dir, "location_to_area.json"), "r") as file:
    location_to_area = json.load(file)

tuna_location_to_area = location_to_area["Tuna"]

In [79]:
stock_assessments = pd.read_excel(
    os.path.join(clean_data_dir, "stock_assessments.xlsx")
)

primary_key = ["Area", "ASFIS Scientific Name", "Location"]

In [80]:
stock_landings = pd.merge(stock_landings, stock_assessments, on=primary_key)

In [87]:
sofia_landings = pd.read_excel(os.path.join(clean_data_dir, "sofia_landings.xlsx"))

In [100]:
sofia_landings["ASFIS Name"] = sofia_landings["ASFIS Scientific Name"].map(
    scientific_to_name
)
sofia_landings["Status"] = sofia_landings["Status"].map(lambda x: {"F": "M"}.get(x, x))
sofia_landings = sofia_landings[sofia_landings["Status"].isin(["U", "M", "O"])]

In [94]:
sofia_landings[
    (sofia_landings["Area"] == 71)
    & (sofia_landings[2021] > 0)
    & (
        sofia_landings["ASFIS Name"].apply(
            lambda x: isinstance(x, str) and "tuna" in x.lower()
        )
    )
]

,Area,ASFIS Scientific Name,Status,Location,Proxy,1950,1951,1952,1953,1954,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,ASFIS Name
691,71,"Auxis thazard, A. rochei",O,NaN,"Auxis thazard, A. rochei",0.0,0.0,0.0,0.0,0.0,...,143205.00,143107.00,146251.00,136646.00,124666.00,114219.00,113621.22,112225.13,95674.40,Frigate and bullet tunas
693,71,Katsuwonus pelamis,F,NaN,Katsuwonus pelamis,42083.0,52226.0,44300.0,33608.0,43353.0,...,1693228.72,1784540.57,1608574.56,1645095.51,1472355.94,1694634.98,1860238.25,1598656.36,1582249.18,Skipjack tuna
695,71,Thunnus albacares,NaN,NaN,Thunnus albacares,8919.0,9395.0,9901.0,10440.0,11013.0,...,502002.04,544631.96,537545.50,602780.54,640140.29,635418.75,616655.44,682611.42,691271.32,Yellowfin tuna


In [106]:
sofia_landings[
    (sofia_landings["Area"] == 51)
    & (sofia_landings[2021] > 0)
    & (
        sofia_landings["ASFIS Name"].apply(
            lambda x: isinstance(x, str) and "tuna" in x.lower()
        )
    )
]

,Area,ASFIS Scientific Name,Status,Location,Proxy,1950,1951,1952,1953,1954,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,ASFIS Name
451,51,Thunnus obesus,O,NaN,Thunnus obesus,16.0,17.0,10.0,10.0,602.0,...,67767.11,55383.45,60232.79,56074.61,60958.69,67435.98,54257.57,59879.02,68121.05,Bigeye tuna
454,51,Katsuwonus pelamis,O,NaN,Katsuwonus pelamis,8968.0,8965.0,8959.0,10215.0,10244.0,...,262320.15,275034.81,269376.10,353065.12,391916.13,468731.73,410541.34,385963.83,469550.63,Skipjack tuna
456,51,Thunnus albacares,O,NaN,Thunnus albacares,2642.0,2318.0,2216.0,2422.0,8209.0,...,320486.27,330206.88,341085.68,371050.16,363355.98,363819.21,363609.96,355889.67,340002.48,Yellowfin tuna


In [103]:
compute_weighted_percentages(sofia_landings, landings_key=2021)

Total Landings                                         \
                 U (Mt)   MSF (Mt)     O (Mt) Sustainable (Mt)   
Area                                                             
21             0.044029   0.901466   0.108179         0.945495   
27             0.188094   5.317797   1.227355         5.505891   
31             0.440744   0.176883   0.049920         0.617627   
34             0.000000   0.584828   1.015532         0.584828   
37             0.000000   0.341623   0.408034         0.341623   
41             0.008877   0.967470   0.356080         0.976347   
47             0.000000   0.889748   0.286781         0.889748   
51             0.082202   0.691112   1.051858         0.773314   
57             0.000000   1.352901   0.529245         1.352901   
61             1.135233   2.564626   4.183096         3.699859   
67             1.159680   1.145306   0.185140         2.304986   
71             0.514889   3.229382   0.994543         3.744270   
77             0.000345   0.112924   0.000536         0.113269   
81             0.000000   0.234466   0.006421         0.234466   
87             0.000000   7.165144   1.480656         7.165144   
Tuna           4.121500   0.839243   0.035857         4.960743   
Global         7.695594  26.514919  11.919231        34.210513   
48,58,88       0.000000   0.000000   0.000000         0.000000   

                            Weighted % by Landings                        \
         Unsustainable (Mt)                  U (%)    MSF (%)      O (%)   
Area                                                                       
21                 0.108179               4.178654  85.554547  10.266799   
27                 1.227355               2.793516  78.978199  18.228285   
31                 0.049920              66.024431  26.497460   7.478109   
34                 1.015532               0.000000  36.543538  63.456462   
37                 0.408034               0.000000  45.570579  54.429421   
41                 0.356080               0.666242  72.609593  26.724165   
47                 0.286781               0.000000  75.624827  24.375173   
51                 1.051858               4.503781  37.865606  57.630614   
57                 0.529245               0.000000  71.880772  28.119228   
61                 4.183096              14.401111  32.533814  53.065075   
67                 0.185140              46.571138  45.993902   7.434960   
71                 0.994543              10.865356  68.147478  20.987166   
77                 0.000536               0.302952  99.225917   0.471130   
81                 0.006421               0.000000  97.334591   2.665409   
87                 1.480656               0.000000  82.874277  17.125723   
Tuna               0.035857              82.486089  16.796277   0.717635   
Global            11.919231              16.682498  57.479007  25.838495   
48,58,88           0.000000                    NaN        NaN        NaN   

                                            
         Sustainable (%) Unsustainable (%)  
Area                                        
21             89.733201         10.266799  
27             81.771715         18.228285  
31             92.521891          7.478109  
34             36.543538         63.456462  
37             45.570579         54.429421  
41             73.275835         26.724165  
47             75.624827         24.375173  
51             42.369386         57.630614  
57             71.880772         28.119228  
61             46.934925         53.065075  
67             92.565040          7.434960  
71             79.012834         20.987166  
77             99.528870          0.471130  
81             97.334591          2.665409  
87             82.874277         17.125723  
Tuna           99.282365          0.717635  
Global         74.161505         25.838495  
48,58,88             NaN               NaN

In [104]:
sofia_landings_fix = sofia_landings[
    ~(
        (sofia_landings["Area"] == 71)
        & (sofia_landings["ASFIS Scientific Name"] == "Katsuwonus pelamis")
    )
]

In [105]:
compute_weighted_percentages(sofia_landings_fix, landings_key=2021)

Total Landings                                         \
                 U (Mt)   MSF (Mt)     O (Mt) Sustainable (Mt)   
Area                                                             
21             0.044029   0.901466   0.108179         0.945495   
27             0.188094   5.317797   1.227355         5.505891   
31             0.440744   0.176883   0.049920         0.617627   
34             0.000000   0.584828   1.015532         0.584828   
37             0.000000   0.341623   0.408034         0.341623   
41             0.008877   0.967470   0.356080         0.976347   
47             0.000000   0.889748   0.286781         0.889748   
51             0.082202   0.691112   1.051858         0.773314   
57             0.000000   1.352901   0.529245         1.352901   
61             1.135233   2.564626   4.183096         3.699859   
67             1.159680   1.145306   0.185140         2.304986   
71             0.514889   1.647132   0.994543         2.162021   
77             0.000345   0.112924   0.000536         0.113269   
81             0.000000   0.234466   0.006421         0.234466   
87             0.000000   7.165144   1.480656         7.165144   
Tuna           4.121500   0.839243   0.035857         4.960743   
Global         7.695594  24.932670  11.919231        32.628264   
48,58,88       0.000000   0.000000   0.000000         0.000000   

                            Weighted % by Landings                        \
         Unsustainable (Mt)                  U (%)    MSF (%)      O (%)   
Area                                                                       
21                 0.108179               4.178654  85.554547  10.266799   
27                 1.227355               2.793516  78.978199  18.228285   
31                 0.049920              66.024431  26.497460   7.478109   
34                 1.015532               0.000000  36.543538  63.456462   
37                 0.408034               0.000000  45.570579  54.429421   
41                 0.356080               0.666242  72.609593  26.724165   
47                 0.286781               0.000000  75.624827  24.375173   
51                 1.051858               4.503781  37.865606  57.630614   
57                 0.529245               0.000000  71.880772  28.119228   
61                 4.183096              14.401111  32.533814  53.065075   
67                 0.185140              46.571138  45.993902   7.434960   
71                 0.994543              16.311690  52.181184  31.507127   
77                 0.000536               0.302952  99.225917   0.471130   
81                 0.006421               0.000000  97.334591   2.665409   
87                 1.480656               0.000000  82.874277  17.125723   
Tuna               0.035857              82.486089  16.796277   0.717635   
Global            11.919231              17.275031  55.968736  26.756233   
48,58,88           0.000000                    NaN        NaN        NaN   

                                            
         Sustainable (%) Unsustainable (%)  
Area                                        
21             89.733201         10.266799  
27             81.771715         18.228285  
31             92.521891          7.478109  
34             36.543538         63.456462  
37             45.570579         54.429421  
41             73.275835         26.724165  
47             75.624827         24.375173  
51             42.369386         57.630614  
57             71.880772         28.119228  
61             46.934925         53.065075  
67             92.565040          7.434960  
71             68.492873         31.507127  
77             99.528870          0.471130  
81             97.334591          2.665409  
87             82.874277         17.125723  
Tuna           99.282365          0.717635  
Global         73.243767         26.756233  
48,58,88             NaN               NaN